Import necessary libraries:

In [1]:
import sys
sys.path.append("..")
from data.preparer import load_news_dataset
from datetime import datetime
from snorkel.labeling import filter_unlabeled_dataframe
from snorkel.labeling import labeling_function
from snorkel.labeling import LabelModel
from snorkel.labeling import LFAnalysis
from snorkel.labeling import PandasLFApplier
from analyzer import train_model
import re
import pandas as pd
stat_history = pd.DataFrame()
import nltk
nltk.download("punkt")
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to /Users/sara/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/sara/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
# Unzip the data. (Don't worry about this, it should be already unzipped.)
# Replace PASSWORD with the password to unzip the data, or download it directly from Kaggle.

#!unzip -P PASSWORD data/data.zip

In [3]:
# load the data
df_train, df_dev, df_valid, df_test = load_news_dataset()
print("{} training examples".format(len(df_train)))
print("{} development examples".format(len(df_dev)))
print("{} validation examples".format(len(df_valid)))
print("{} test examples".format(len(df_dev)))

477 training examples
400 development examples
100 validation examples
400 test examples


/Users/sara/Documents/GitHub/snorkel-notebooks/venv/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


# Snorkel Tutorial

## News forum classification

### You will work with a subset of the 20 NewsGroup dataset. 
The texts shown are from one of two forums:
 1. Computer Electronics (Label 0)
 2. Gun Politics Forum (Label 1)
Your job is to create a training data set to classify texts as belonging to one of these two forums.

You will do this by writing labeling functions mapping text to 0 (ELECTRONICS), 1 (GUNS), or -1 (ABSTAIN or no label).
These functions will be aggregated by Snorkel to create training data from unlabeled examples.

You can evaluate your progress based on the coverage and f1 score of your label model, or by training a logistic regression classifier on the data and evaluating the test result.


In [4]:
# Start the timer!
stat_history.append({"time": datetime.now(), "num_lfs": 0}, ignore_index=True)

,num_lfs,time
0,0.0,2020-02-01 15:03:26.345609


Define the labels for this task:

In [5]:
ABSTAIN = -1
ELECTRONICS = 0
GUNS = 1

# Ready to Roll
Let's look at some positive and negative examples.

In [6]:
# rerun this cell to get a new sample
print("ELECTRONICS")
display(df_dev[df_dev.label==ELECTRONICS].sample(5))
print("GUNS")
display(df_dev[df_dev.label==GUNS].sample(5))

ELECTRONICS


,text,label
566,Has anybody out there used/tested these new Fl...,0
211,": HI, I was wondering if anyone would be able ...",0
500,\nGo-Video machines used in HQ2 mode will copy...,0
517,I have an Adcom GFA-555 that I got in 1985. T...,0
209,There has been quite a bit of discussion about...,0


GUNS


,text,label
288,Ever since the siege at Waco started the FBI s...,1
252,\nI knew that Cutie would sell us out. Full-b...,1
327,\nNot to mention last year's Weaver affair.\n\...,1
212,"\nHowever, don't forget that the police in Chi...",1
520,\nThere is a (likely) veto proof majority in t...,1


## Writing Labeling Functions

__Your task for this tutorial is to write 5 labeling functions.__

Feel free to consult the internet or ask your experiment leader.

*(For the real task, you will be given 30 minutes. You will still be allowed to use the internet in this phase, but not ask your experiment leader.)*

Your function should take x as an input and output ELECTRONICS, GUNS, or ABSTAIN. 

In [7]:
@labeling_function()
def lf0(x):
    return GUNS if "firearm" in x.text.lower() else ABSTAIN

Your turn! try writing a function or editing the one above.

If you want, you can write helper functions to reuse. For example:

In [8]:
# example helper function 
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
def sentiment(text):
    return sia.polarity_scores(text)["compound"]

In [9]:
@labeling_function()
def lf1(x):
    return ELECTRONICS if "tesla" in x.text.lower() else ABSTAIN

In [10]:
@labeling_function()
def lf2(x):
    count = len(re.findall("power", x.text.lower())) 
    return ELECTRONICS if count > 2 else GUNS 

In [11]:
@labeling_function()
def lf3(x):
    return ABSTAIN

In [12]:
@labeling_function()
def lf4(x):
    return ABSTAIN

Test your function (optional)

In [13]:
from types import SimpleNamespace

def test_func(lf, example_text):
    x = SimpleNamespace(text=example_text)
    return lf(x)

In [14]:
test_func(lf2, "I've got a powerful powerful tesla coil")

1

## Applying Functions
This is how we obtain training labels, by training a model to combine the outputs of the noisy labeling functions.
`L_train` and `L_dev` are matrices representing the label returned by each labeling function for each example in the training and development sets.

In [15]:
# Make sure this list has all your labeling functions
lfs = [lf0, lf1, lf2, lf3, lf4]

# Apply the LFs to the unlabeled training data, and the development data
applier = PandasLFApplier(lfs)
L_train = applier.apply(df_train)
L_dev = applier.apply(df_dev)

100%|██████████| 400/400 [00:00<00:00, 14769.46it/s]


Train the snorkel model to combine these noisy labels.

In [16]:
# Train the label model and compute the training labels
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, n_epochs=500, log_freq=50, seed=123)
df_train["pred_label"] = label_model.predict_proba(L=L_train)[:,0]
probs_train = df_train["pred_label"]

# record intermediate results
# Don't worry about this code block, we just store some metrics to keep track of your progress.
Y_dev = df_dev.label.values
stats = label_model.score(L=L_dev, Y=Y_dev, metrics=["f1", "precision", "recall"])
df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(
            X=df_train, y=probs_train, L=L_train)
stats["training_label_coverage"] = len(probs_train_filtered)/len(probs_train)
stats["training_label_size"] = len(probs_train_filtered)
stats["time"] = datetime.now()
stat_history = stat_history.append(stats, ignore_index=True)

In [17]:
# let's see some examples of aggregated (probabilistic) labels!
# re run this cell for new examples
display(df_train.sample(5))

,text,pred_label
1008,"Yes, I know it sounds crazy. Call it an urge. ...",0.388506
1019,"\nAs a general rule, no relay will cleanly swi...",0.388506
618,"According to WNCI 97.9 FM radio this morning, ...",0.339520
1091,"\n\tIf alcohol were again banned today, it wou...",0.388506
669,Here is a copy of my first update on the Randy...,0.388506


## View Unlabeled Examples
You can use these to brainstorm new labeling functions. You may try filtering or sorting them in other ways.

If you get a `ValueError: a must be greater than 0 unless no samples are taken`, this means all your training examples are labeled by at least one LF.

In [18]:
# You can filter for unlabeled data
try:
    df_unlabeled = df_train[~df_train.index.isin(df_train_filtered.index)]
    display(df_unlabeled.sample(5))
except ValueError as e:
    print("ValueError: ")
    print(e)
    label_sums = (L_train != -1).sum(axis=1)
    print("\nExamples with lowest coverage: ({})".format(min(label_sums)))
    display(df_train[label_sums == min(label_sums)].head())

ValueError: 
a must be greater than 0 unless no samples are taken

Examples with lowest coverage: (1)


,text,pred_label
615,"I had a GE Emerson 13"" color TV for about 3 ye...",0.388506
616,\nAre you switching high level signals or low ...,0.388506
617,\n\n\n\nAnd they work especially well when the...,0.388506
619,\nThey do. According the the Los Angeles Polic...,0.388506
620,Doesn't Motorola AMCU have something on the BB...,0.388506


## Analyze Results
Evaluate the accuracy of the estimated training labels and development set labels (based on ground truth).

`Polarity` describes the set of outputs of each function, not including `ABSTAIN (-1)`.
For example, a function that returns `ABSTAIN` or `GUNS` has polarity `[1]`

In [19]:
train_analysis = LFAnalysis(L=L_train, lfs=lfs).lf_summary()
display("Training set results:", train_analysis)

'Training set results:'

,j,Polarity,Coverage,Overlaps,Conflicts
lf0,0,[1],0.088050,0.088050,0.000000
lf1,1,[0],0.002096,0.002096,0.002096
lf2,2,"[0, 1]",1.000000,0.090147,0.002096
lf3,3,[],0.000000,0.000000,0.000000
lf4,4,[],0.000000,0.000000,0.000000


In [20]:
Y_dev = df_dev.label.values
dev_analysis = LFAnalysis(L=L_dev, lfs=lfs).lf_summary(Y=Y_dev)
display("Dev set results:", dev_analysis)

'Dev set results:'

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
lf0,0,[1],0.0675,0.0675,0.000,27,0,1.00
lf1,1,[0],0.0050,0.0050,0.005,2,0,1.00
lf2,2,"[0, 1]",1.0000,0.0725,0.005,190,0,0.48
lf3,3,[],0.0000,0.0000,0.000,0,0,0.00
lf4,4,[],0.0000,0.0000,0.000,0,0,0.00


## Save the Model
When you have finished

In [21]:
label_model.save("snorkel_tutorial_lfmodel.pkl")
stat_history.to_csv("snorkel_tutorial_statistics_history.csv")

## Train Model
We can train a simple bag of words model on these labels, and see test accuracy.

(This step may take a while).

In [22]:
train_model(label_model, df_train, df_valid, df_test, L_train)

AttributeError: 'DataFrame' object has no attribute 'label'